In [3]:
import tensorflow as tf

x = tf.Variable(3, name = "x")
y = tf.Variable(4, name = "y")
f = x * x * y + 2

In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

38


In [3]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result  = f.eval()
print(result)

38


变量初始化

In [4]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

## 管理图

In [5]:
x1 = tf.Variable(1)
print(x1.graph is tf.get_default_graph())

graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
print(x1.graph)    
x2.graph

True


## 计算节点的生命周期 Lifecycle of a Node Value


In [6]:
w = tf.constant(3)
x = w + 2
y = x * 9
z = x * y

with tf.Session() as sess:
    print(z.eval())

225


## 用TF解决线性回归问题 Linear Regression with TensorFlow


In [5]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
# θ = (XT · X)–1 · XT · y
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
with tf.Session() as sess:
    theta_value = theta.eval()


## 梯度下降 
### Manually Computing the Gradients

需要特征缩放，使用sklearn的StandardScaler来特征缩放

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]


n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2 / m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate*gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE =  9.555813
Epoch 100 MSE =  0.7359047
Epoch 200 MSE =  0.5772647
Epoch 300 MSE =  0.56097335
Epoch 400 MSE =  0.55086106
Epoch 500 MSE =  0.5435743
Epoch 600 MSE =  0.5383
Epoch 700 MSE =  0.53448063
Epoch 800 MSE =  0.5317129
Epoch 900 MSE =  0.52970576


## 使用 自动微分法autodiff


In [9]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
reset_graph()

n_epochs = 2000
learning_rate = 0.01

X= tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -0.9, 0.9, seed=42))
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

gradients = tf.gradients(mse, [theta])[0]
g1 = tf.gradients(mse, [theta])

training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            print(sess.run(gradients))
            print(sess.run(g1))
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 8.6325445
[[-4.4384294 ]
 [-2.6223013 ]
 [-0.7782153 ]
 [ 0.10779657]
 [ 1.2499629 ]
 [ 1.7100307 ]
 [ 0.55459267]
 [-0.5326979 ]
 [ 1.1299009 ]]
[array([[-4.4384294 ],
       [-2.6223013 ],
       [-0.7782153 ],
       [ 0.10779657],
       [ 1.2499629 ],
       [ 1.7100307 ],
       [ 0.55459267],
       [-0.5326979 ],
       [ 1.1299009 ]], dtype=float32)]
Epoch 100 MSE = 0.7095024
[[-0.5886214 ]
 [-0.34870744]
 [-0.02075823]
 [ 0.01459869]
 [ 0.11869746]
 [ 0.18805727]
 [ 0.06130667]
 [ 0.09306485]
 [ 0.11236905]]
[array([[-0.5886214 ],
       [-0.34870744],
       [-0.02075823],
       [ 0.01459869],
       [ 0.11869746],
       [ 0.18805727],
       [ 0.06130667],
       [ 0.09306485],
       [ 0.11236905]], dtype=float32)]
Epoch 200 MSE = 0.5682746
[[-0.07806297]
 [-0.04300554]
 [ 0.01235042]
 [ 0.01654888]
 [-0.00367407]
 [ 0.02914782]
 [ 0.00586761]
 [ 0.07261223]
 [ 0.07254743]]
[array([[-0.07806297],
       [-0.04300554],
       [ 0.01235042],
       [ 0.016548

## Using an Optimizer


In [10]:
reset_graph()
n_epochs = 2000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -0.9, 0.9, seed=42))
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer  = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 8.6325445
Epoch 100 MSE = 0.7095024
Epoch 200 MSE = 0.5682746
Epoch 300 MSE = 0.55664164
Epoch 400 MSE = 0.54944104
Epoch 500 MSE = 0.54396886
Epoch 600 MSE = 0.53976834
Epoch 700 MSE = 0.5365259
Epoch 800 MSE = 0.5340092
Epoch 900 MSE = 0.5320459
Epoch 1000 MSE = 0.5305077
Epoch 1100 MSE = 0.52929586
Epoch 1200 MSE = 0.52833605
Epoch 1300 MSE = 0.52757233
Epoch 1400 MSE = 0.5269619
Epoch 1500 MSE = 0.5264731
Epoch 1600 MSE = 0.5260783
Epoch 1700 MSE = 0.52576005
Epoch 1800 MSE = 0.5255015
Epoch 1900 MSE = 0.52529186
Best theta:
[[ 2.0685525e+00]
 [ 8.5579723e-01]
 [ 1.2668240e-01]
 [-3.0933258e-01]
 [ 3.3943361e-01]
 [-1.9577104e-03]
 [-4.0555313e-02]
 [-8.1749099e-01]
 [-7.9091966e-01]]


In [11]:
reset_graph()
n_epochs = 2000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -0.9, 0.9, seed=42))
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer  = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 8.6325445
Epoch 100 MSE = 2.4318814
Epoch 200 MSE = 0.9845251
Epoch 300 MSE = 0.608887
Epoch 400 MSE = 0.5355876
Epoch 500 MSE = 0.52558774
Epoch 600 MSE = 0.5244939
Epoch 700 MSE = 0.5243536
Epoch 800 MSE = 0.52432764
Epoch 900 MSE = 0.5243219
Epoch 1000 MSE = 0.5243211
Epoch 1100 MSE = 0.52432114
Epoch 1200 MSE = 0.52432066
Epoch 1300 MSE = 0.52432096
Epoch 1400 MSE = 0.52432054
Epoch 1500 MSE = 0.5243205
Epoch 1600 MSE = 0.52432054
Epoch 1700 MSE = 0.5243205
Epoch 1800 MSE = 0.52432054
Epoch 1900 MSE = 0.52432054
Best theta:
[[ 2.068555  ]
 [ 0.8296182 ]
 [ 0.1187515 ]
 [-0.2655253 ]
 [ 0.30569488]
 [-0.00450304]
 [-0.03932624]
 [-0.8998872 ]
 [-0.87054247]]


## 将数据提供给训练算法Feeding data to the training algorithm

使用 placeholder

In [12]:
n_epochs = 2000
learning_rate = 0.01

reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -0.9, 0.9, seed=42))

y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size)) # 取整

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # 
    indices = np.random.randint(m, size=batch_size)  # 
    X_batch = scaled_housing_data_plus_bias[indices] # 
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            
            if batch_index == int(m / batch_size):
                print("Epoch", epoch, "mse=", sess.run(mse, feed_dict={X: X_batch, y: y_batch}))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
        best_theta = theta.eval()

print("Best theta:")
print(best_theta)


Epoch 0 mse= 0.48219642
Epoch 1 mse= 0.65663743
Epoch 2 mse= 0.48741966
Epoch 3 mse= 0.6732495
Epoch 4 mse= 0.42330834
Epoch 5 mse= 0.49597272
Epoch 6 mse= 0.34872764
Epoch 7 mse= 0.39867553
Epoch 8 mse= 0.6930215
Epoch 9 mse= 0.37455258
Best theta:
[[ 2.0703187 ]
 [ 0.862098  ]
 [ 0.12252025]
 [-0.3086222 ]
 [ 0.3818825 ]
 [ 0.00437853]
 [-0.01234637]
 [-0.83555514]
 [-0.8045839 ]]


## 保存模型

In [22]:
import os
reset_graph()
n_epochs = 2000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -0.9, 0.9, seed=42))
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer  = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            
        sess.run(training_op)
        save_path = saver.save(sess, os.path.join(os.getcwd(), '../models/train_002.ckpt'))
    
    best_theta = theta.eval()
    save_path = saver.save(sess, os.path.join(os.getcwd(), '../models/train_002.ckpt'))
print("Best theta:")
print(best_theta)

Epoch 0 MSE = 8.6325445
Epoch 100 MSE = 2.4318814
Epoch 200 MSE = 0.9845251
Epoch 300 MSE = 0.608887
Epoch 400 MSE = 0.5355876
Epoch 500 MSE = 0.52558774
Epoch 600 MSE = 0.5244939
Epoch 700 MSE = 0.5243536
Epoch 800 MSE = 0.52432764
Epoch 900 MSE = 0.5243219
Epoch 1000 MSE = 0.5243211
Epoch 1100 MSE = 0.52432114
Epoch 1200 MSE = 0.52432066
Epoch 1300 MSE = 0.52432096
Epoch 1400 MSE = 0.52432054
Epoch 1500 MSE = 0.5243205
Epoch 1600 MSE = 0.52432054
Epoch 1700 MSE = 0.5243205
Epoch 1800 MSE = 0.52432054
Epoch 1900 MSE = 0.52432054
Best theta:
[[ 2.068555  ]
 [ 0.8296182 ]
 [ 0.1187515 ]
 [-0.2655253 ]
 [ 0.30569488]
 [-0.00450304]
 [-0.03932624]
 [-0.8998872 ]
 [-0.87054247]]


In [23]:
#saver = tf.train.import_meta_graph("../models/train_002.ckpt.meta")
#theta = tf.get_default_graph().get_tensor_by_name("theta:0")

with tf.Session() as sess:
    saver.restore(sess, "../models/train_002.ckpt")
    best_theta_restored = theta.eval()
    print(best_theta_restored)

INFO:tensorflow:Restoring parameters from ../models/train_002.ckpt
[[ 2.068555  ]
 [ 0.8296182 ]
 [ 0.1187515 ]
 [-0.2655253 ]
 [ 0.30569488]
 [-0.00450304]
 [-0.03932624]
 [-0.8998872 ]
 [-0.87054247]]


## 使用TensorBoard
 

In [24]:
reset_graph()

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [28]:
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
    best_theta = theta.eval()
    print(best_theta)

[[ 2.0703337 ]
 [ 0.8637145 ]
 [ 0.12255151]
 [-0.31211874]
 [ 0.38510373]
 [ 0.00434168]
 [-0.01232954]
 [-0.83376896]
 [-0.8030471 ]]


In [29]:
file_writer.close()

## 命名空间

In [34]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

file_writer.flush()
file_writer.close()
print("Best theta:")
print(best_theta)

Best theta:
[[ 2.0703337 ]
 [ 0.8637145 ]
 [ 0.12255151]
 [-0.31211874]
 [ 0.38510373]
 [ 0.00434168]
 [-0.01232954]
 [-0.83376896]
 [-0.8030471 ]]


## 共享变量
using name scopes:

这种方法就是 共用一个relu空间，每次循环创建一个relu变量

In [10]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
reset_graph()
def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name = "bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")
n_features = 3   
X = tf.placeholder(tf.float32, shape=(None, n_features), name = "X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                               initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu6", tf.get_default_graph())
file_writer.close()

这种方法就是 共用第一个relu空间，每次循环创建一个relu变量

In [11]:
def relu(X):
    threshold = tf.get_variable("threshold", shape=(),
                               initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")
file_writer = tf.summary.FileWriter("logs/relu9", tf.get_default_graph())
file_writer.close()

ValueError: Variable relu/threshold already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-10-ec588a0f4f97>", line 18, in <module>
    initializer=tf.constant_initializer(0.0))
  File "/home/bladeray/anaconda3/envs/tf/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2862, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/bladeray/anaconda3/envs/tf/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2802, in run_ast_nodes
    if self.run_code(code, result):
